# Protein Folding with Pyrosetta

In this tutorial, we will fold a protein structure using a very simple algorithm in PyRosetta, and compare the folded structure with the solved crystal structure of the protein.

### Importing relevant libraries

We begin by importing the relevant libraries from Python. If running the following cell produces any errors or warnings, make sure you have followed all the steps in the "Setting up Pyrosetta" section.

In [1]:
import os
import glob
import shutil
import pandas as pd
import nglview as ngl
import pyrosetta as prs
prs.init()
from pyrosetta import rosetta

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.mac r213 2019.10+release.fd1bdffb01b fd1bdffb01b7866da84942b9bf1b06e96270656e http://www.pyrosetta.org 2019-03-05T15:28:05
core.init: command: PyRosetta -ex1 -ex2aro -database /anaconda3/envs/pyrosetta/lib/python3.6/site-packages/pyrosetta-2019.10+release.fd1bdffb01b-py3.6-macosx-10.7-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=1149888667 seed_offset=0 real_seed=1149888667
core.init.random: RandomGenerator:init: Normal mode, seed=1149888667 RG_type=mt19937


### Setting up score functions that will be used across parts

In [2]:
scorefxn_low = prs.create_score_function('score3')
scorefxn_high = prs.get_fa_scorefxn()

basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/env_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cbeta_den.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/pair_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cenpack_log.txt
basic.io.database: Database file opened: scoring/score_functions/SecondaryStructurePotential/phi.theta.36.HS.resmooth
basic.io.database: Database file opened: scoring/score_functions/SecondaryStructurePotential/phi.theta.36.SS.resmooth
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: F

### Loading the native (solved crystal) structure

In [40]:
native_pose = prs.pose_from_pdb('data/1BL0/1BL0_chainA.pdb')

core.import_pose.import_pose: File 'data/1BL0/1BL0_chainA.pdb' automatically determined to be of type PDB


We can check the amino acid sequence of the structure with a very simple command.

In [41]:
native_pose.sequence()

'DAITIHSILDWIEDNLESPLSLEKVSERSGYSKWHLQRMFKKETGHSLGQYIRSRKMTEIAQKLKESNEPILYLAERYGFESQQTLTRTFKNYFDVPPHKYRMTNMQGESRFLHPL'

We can also assign the correct secondary structure.


In [42]:
DSSP = prs.rosetta.protocols.moves.DsspMover()
DSSP.apply(native_pose)    # populates the pose's Pose.secstruct

protocols.DsspMover: LHHHHHHHHHHHHLLLLLLLLLHHHHHHLLLLHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHLLLLHHHHHHHHHHHHLLLHHHHHLLLLLLLLLLLLLL


#### Hint: The amino acids are also called residues!

Let's view more information about the first residue of the protein.

In [43]:
print(native_pose.residue(1))

Residue 1: ASP:NtermProteinFull (ASP, D):
Base: ASP
 Properties: POLYMER PROTEIN CANONICAL_AA LOWER_TERMINUS SC_ORBITALS POLAR CHARGED NEGATIVE_CHARGE METALBINDING ALPHA_AA L_AA
 Variant types: LOWER_TERMINUS_VARIANT
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O   1H   2H   3H    HA 
 Side-chain atoms:  CB   CG   OD1  OD2 1HB  2HB 
Atom Coordinates:
   N  : 0.229, 36.012, 74.172
   CA : 0.041, 35.606, 75.594
   C  : -0.096, 36.849, 76.498
   O  : -0.951, 36.895, 77.382
   CB : 1.225, 34.718, 76.092
   CG : 2.159, 34.156, 74.999
   OD1: 1.688, 33.361, 74.151
   OD2: 3.378, 34.497, 75.007
  1H  : 1.056, 35.74, 73.68
  2H  : -0.43, 35.723, 73.478
  3H  : 0.251, 36.981, 73.928
   HA : -0.884, 35.037, 75.696
  1HB : 1.839, 35.199, 76.854
  2HB : 0.67, 33.892, 76.539
Mirrored relative to coordinates in ResidueType: FALSE



In [44]:
pose = prs.pose_from_sequence(native_pose.sequence())
test_pose = prs.Pose()
test_pose.assign(pose)
test_pose.pdb_info().name('Linearized Pose')

In [45]:
view = ngl.show_rosetta(test_pose)
view.add_ball_and_stick()
view  # zoom in to see the atoms!

NGLWidget()

## Defining movers to switch from full-atom to centroid representation

We will be using the centroid representation to perform rough and fast scoring in the initial stages of the folding algorithm. Later on, we will switch to the full atom represenation to do accurate minimization and get the final structures.

In [48]:
to_centroid = prs.SwitchResidueTypeSetMover('centroid')
to_full_atom = prs.SwitchResidueTypeSetMover('fa_standard')

In [49]:
to_full_atom.apply(test_pose)
print('Full Atom Score:', scorefxn_high(test_pose))
to_centroid.apply(test_pose)
print('Centroid Score:', scorefxn_low(test_pose))

Full Atom Score: 46742.26828801393
Centroid Score: 454.12286307835507


## Q: Visualize the centroid-only structure and see the difference with the full atom that we visualized above? Print again the information for the first residue and compare.

In [83]:
# here write the code to visualize the centroid only structure and print the information of the 1st residue

## Setting up the folding algorithm

In [52]:
# Loading the files with the pre-computed fragmets
long_frag_filename = 'data/1BL0/9_fragments.txt'
long_frag_length = 9
short_frag_filename = 'data/1BL0/3_fragments.txt'
short_frag_length = 3

# Defining parameters of the folding algorithm
long_inserts=5  # How many 9-fragment pieces to insest during the search
short_inserts=10 # How many 3-fragment pieces to insest during the search

kT = 3.0 # Simulated Annealing temperature
cycles = 1000 # How many cycles of Monte Carlo search to run
jobs = 50 # How many trajectories in parallel to compute.
job_output = 'outputs/1BL0/decoy' # The prefix of the filenames to store the results

#### Loading the fragmets

In [53]:
movemap = prs.MoveMap()
movemap.set_bb(True)

fragset_long = rosetta.core.fragment.ConstantLengthFragSet(long_frag_length, long_frag_filename)
long_frag_mover = rosetta.protocols.simple_moves.ClassicFragmentMover(fragset_long, movemap)

fragset_short = rosetta.core.fragment.ConstantLengthFragSet(short_frag_length, short_frag_filename)
short_frag_mover = rosetta.protocols.simple_moves.ClassicFragmentMover(fragset_short, movemap)

insert_long_frag = prs.RepeatMover(long_frag_mover, long_inserts)
insert_short_frag = prs.RepeatMover(short_frag_mover, short_inserts)

core.fragments.ConstantLengthFragSet: finished reading top 200 9mer fragments from file data/1BL0/9_fragments.txt
core.fragments.ConstantLengthFragSet: finished reading top 200 3mer fragments from file data/1BL0/3_fragments.txt


## Q: How many 9-mer and 3-mer fragmets do we have in our database?

#### Setting up the MonteCarlo search

In [57]:
# Making sure the structure is in centroid-only mode for the search
test_pose.assign(pose)
to_centroid.apply(test_pose)

In [58]:
# Defining what sequence of actions to do between each scoring step
folding_mover = prs.SequenceMover()
folding_mover.add_mover(insert_long_frag)
folding_mover.add_mover(insert_short_frag)

In [59]:
mc = prs.MonteCarlo(test_pose, scorefxn_low, kT)
trial = prs.TrialMover(folding_mover, mc)

In [60]:
# Setting up how many cycles of search to do in each trajectory
folding = prs.RepeatMover(trial, cycles)

#### Setting up the relax mover for the final stage

In [65]:
fast_relax_mover = prs.rosetta.protocols.relax.FastRelax(scorefxn_high)

protocols.relax.FastRelax: Looking for default.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /anaconda3/envs/pyrosetta/lib/python3.6/site-packages/pyrosetta-2019.10+release.fd1bdffb01b-py3.6-macosx-10.7-x86_64.egg/pyrosetta/database/sampling/relax_scripts/default.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: ramp_repack_min 0.02  0.01     1.0
protocols.relax.RelaxScriptManager: ramp_repack_min 0.250 0.01     0.5
protocols.relax.RelaxScriptManager: ramp_repack_min 0.550 0.01     0.0
protocols.relax.RelaxScriptManager: ramp_repack_min 1     0.00001  0.0
protocols.relax.RelaxScriptManager: accept_to_best
protocols.relax.RelaxScriptManager: endrepeat


### Running the folding algorithm!

In [66]:
scores = [0] * (jobs + 1)
scores[0] = scorefxn_low(test_pose)

In [67]:
if os.path.isdir(os.path.dirname(job_output)):
    shutil.rmtree(os.path.dirname(job_output), ignore_errors=True)
os.makedirs(os.path.dirname(job_output))
jd = prs.PyJobDistributor(job_output, nstruct=jobs, scorefxn=scorefxn_high)

Working on decoy: outputs/1BL0/decoy_12.pdb


In [68]:
counter = 0 
while not jd.job_complete:
    # a. set necessary variables for the new trajectory
    # -reload the starting pose
    test_pose.assign(pose)
    to_centroid.apply(test_pose)
    # -change the pose's PDBInfo.name, for the PyMOL_Observer
    counter += 1
    test_pose.pdb_info().name(job_output + '_' + str(counter))
    # -reset the MonteCarlo object (sets lowest_score to that of test_pose)
    mc.reset(test_pose)

    #### if you create a custom protocol, you may have additional
    ####    variables to reset, such as kT

    #### if you create a custom protocol, this section will most likely
    ####    change, many protocols exist as single Movers or can be
    ####    chained together in a sequence (see above) so you need
    ####    only apply the final Mover
    # b. apply the refinement protocol
    folding.apply(test_pose)

    ####
    # c. export the lowest scoring decoy structure for this trajectory
    # -recover the lowest scoring decoy structure
    mc.recover_low(test_pose)
    # -store the final score for this trajectory
    # -convert the decoy to fullatom
    # the sidechain conformations will all be default,
    #    normally, the decoys would NOT be converted to fullatom before
    #    writing them to PDB (since a large number of trajectories would
    #    be considered and their fullatom score are unnecessary)
    # here the fullatom mode is reproduced to make the output easier to
    #    understand and manipulate, PyRosetta can load in PDB files of
    #    centroid structures, however you must convert to fullatom for
    #    nearly any other application
    to_full_atom.apply(test_pose)
    fast_relax_mover.apply(test_pose)
    scores[counter] = scorefxn_high(test_pose)
    # -output the fullatom decoy structure into a PDB file
    jd.output_decoy(test_pose)
    # -export the final structure to PyMOL
    test_pose.pdb_info().name(job_output + '_' + str(counter) + '_fa')

Working on decoy: outputs/1BL0/decoy_36.pdb
Working on decoy: outputs/1BL0/decoy_48.pdb
Working on decoy: outputs/1BL0/decoy_46.pdb
Working on decoy: outputs/1BL0/decoy_39.pdb


KeyboardInterrupt: 

### Evaluating the folding results by getting the energies and RMSDs (distnace from the native pose) of each one of the predicted structures

In [69]:
decoy_poses = [prs.pose_from_pdb(f) for f in glob.glob(job_output + '*.pdb')]

core.import_pose.import_pose: File 'outputs/1BL0/decoy_46.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'outputs/1BL0/decoy_36.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'outputs/1BL0/decoy_12.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'outputs/1BL0/decoy_48.pdb' automatically determined to be of type PDB


In [70]:
def align_and_get_rmsds(native_pose, decoy_poses):
    prs.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native_pose)
    rmsds = []
    for p in decoy_poses:
        prs.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(p)
        rmsds += [prs.rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native_pose, p)]
    return rmsds

In [71]:
rmsds = align_and_get_rmsds(native_pose, decoy_poses)

protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 14.7688798)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 14.6700399)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 15.0408467)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 17.8154239)


In [78]:
rmsd_data = []
for i in range(1, len(decoy_poses)):  # print out the job scores
    rmsd_data.append({'structure': decoy_poses[i].pdb_info().name(), 
                      'rmsd': rmsds[i],
                      'energy_score': scores[i]})

In [79]:
rmsd_df = pd.DataFrame(rmsd_data)

In [81]:
rmsd_df.sort_values('rmsd')

,energy_score,rmsd,structure
0,71579.876477,14.670040,outputs/1BL0/decoy_36.pdb
1,71376.772015,15.040847,outputs/1BL0/decoy_12.pdb
2,72489.264360,17.815424,outputs/1BL0/decoy_48.pdb


### Q: Plot the energy_score VS rmsd. Are the lowest energy structures the closest to the native one?